## Supplier Risk MVP for Schmidt Medical Technology 

- Goal is to build a minimal, explainable quantitative risk score per supplier using the customer's ERP export. 
- Inputs: 5 CSVs (orders, suppliers, articles, addresses, indices)
- Outputs: tidy result table with "supplier_id", "country", "sum_of_oder_volume", "industry", "quantitative_ris_score" and some visuals
- Remark: all assumptions & weights are set in the CONFIG cell to ensure fast iteration

In [52]:
# Imports
import pandas as pd
import numpy as np
from dataclasses import asdict
import sys
from pathlib import Path
from src.utils import Config
from copy import deepcopy
import pycountry

sys.path.append(str(Path("..").resolve()))  

# Paths
PROJ = Path("..").resolve() # moves one level up from notebooks folder (parent folder) 
DATA = PROJ / 'data' / 'raw' # portable path to input files
REPORTS = PROJ / 'reports' # results/figures are saved here
REPORTS.mkdir(parents=True, exist_ok=True) # ensures folder exists to ensure figure/results saves

## Step 1: Load & preview CSVs

In [53]:
# Define file paths for all 5 datasets (stored in data/raw/)
FILES = {
    'addresses': DATA / 'addresses.csv',
    'articles': DATA / 'articles.csv',
    'indices': DATA / 'indices.csv',
    'orders': DATA / 'orders.csv',
    'suppliers': DATA / 'suppliers.csv',
}

# Load each existing CSV into a pandas dataframe (df)
# -> key = dataset name (string)
# -> value = corresponding df
dfs = {k: pd.read_csv(v) for k, v in FILES.items() if v.exists()}

# Preview the first 5 rows of each dataset and basic structure info
for name, df in dfs.items():
    print(f"\n--- {name.upper()} ---")
    display(df.head(5))  
    print(df.info())     # column dtypes, non-null counts, memory usage


--- ADDRESSES ---


,supplier_id,country
0,1003,Sweden
1,1084,Germany
2,1076,germany
3,1093,Germany
4,1037,germany


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   supplier_id  117 non-null    int64 
 1   country      117 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.0+ KB
None

--- ARTICLES ---


,Article_ID,Article,Industry
0,"""1000001""",Standardteile aus Aluminium,Metals
1,"""1000002""",Blechteile,metals
2,"""1000003""",Rohmetalle,Metals
3,"""1000004""",Elektronische Standardteile,Electronics
4,"""1000005""",Technische Produkte,electronics


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Article_ID  11 non-null     object
 1   Article     11 non-null     object
 2   Industry    11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes
None

--- INDICES ---


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country_id          249 non-null    object 
 1   human_rights_index  142 non-null    float64
 2   enivronmental_risk  220 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.0+ KB
None

--- ORDERS ---


,order_id,price_per_quantity,quantity,order_value,supplier_id,article_id
0,O00107,470.33,92,43270.36,"""1001""",1000001
1,O00109,241.40,163,39348.20,"""1001""",1000001
2,O00112,357.63,7,2503.41,"""1001""",1000002
3,O00278,865.39,125,108173.75,"""1001""",1000002
4,O00300,889.69,58,51602.02,"""1001""",1000003


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            500 non-null    object 
 1   price_per_quantity  500 non-null    float64
 2   quantity            500 non-null    int64  
 3   order_value         500 non-null    float64
 4   supplier_id         500 non-null    object 
 5   article_id          500 non-null    int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 23.6+ KB
None

--- SUPPLIERS ---


,supplier_id,total_company_revenue,domain,status,certificates_valid
0,1003,1.377955e+09,NaN,inactive,no
1,1084,1.074170e+09,NaN,active,yes
2,1076,9.030552e+08,NaN,active,yes
3,1093,7.450531e+08,NaN,active,yes
4,1037,1.700478e+09,NaN,inactive,yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   supplier_id            106 non-null    int64  
 1   total_company_revenue  106 non-null    float64
 2   domain                 0 non-null      float64
 3   status                 106 non-null    object 
 4   certificates_valid     106 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 4.3+ KB
None


## Step 2: Data Audit

In [54]:
# 1) Primary key (pk) uniqueness check 
# to ensure that pks do not have duplicates (could break joins & aggregations later)
pk_checks = {
    "addresses": "supplier_id", # expect one address per supplier
    "orders": "order_id",       # each order should have unique ID
    "articles": "Article_ID",   # each article should be unique
    "suppliers": "supplier_id", # each supplier should appear once
    "indices": "country_id",    # each country should appear once in indices
}
uniqueness = {}
for name, pk in pk_checks.items():
    dupe = dfs[name][pk].duplicated().sum()
    uniqueness[name] = {"pk": pk, "duplicates": int(dupe)}
uniqueness

{'addresses': {'pk': 'supplier_id', 'duplicates': 17},
 'orders': {'pk': 'order_id', 'duplicates': 0},
 'articles': {'pk': 'Article_ID', 'duplicates': 0},
 'suppliers': {'pk': 'supplier_id', 'duplicates': 6},
 'indices': {'pk': 'country_id', 'duplicates': 0}}

In [55]:
# Check duplicate supplier_ids in suppliers table
suppliers_df = dfs["suppliers"]

# Identify supplier_ids with more than one row
dup_sup_ids = suppliers_df["supplier_id"][suppliers_df["supplier_id"].duplicated()].unique()

# Show all rows for those supplier_ids
suppliers_duplicates = suppliers_df[suppliers_df["supplier_id"].isin(dup_sup_ids)]

display(suppliers_duplicates.sort_values("supplier_id"))

,supplier_id,total_company_revenue,domain,status,certificates_valid
32,1019,12187127.40,NaN,inactive,yes
33,1019,12187127.40,NaN,inactive,yes
34,1019,12187127.40,NaN,inactive,yes
11,1055,8295672.56,NaN,active,yes
12,1055,8295672.56,NaN,active,yes
90,1086,7725679.68,NaN,active,yes
91,1086,7725679.68,NaN,active,yes
92,1086,7725679.68,NaN,active,yes
18,1098,18603697.32,NaN,active,no
19,1098,18603697.32,NaN,active,no


#### Quick interpretation of PK uniqueness check: 
- 17 duplicates in supplier IDs: some suppliers may have multiple addresses (may operate in multiple countries)
- 6 supplier duplicates: each supplier should likely have one record (can only take the first one since other values are the same)

In [56]:
# 2) Referential integrity
# checking whether all the foreign keys in one table have matching primary keays in the related table
# e.g.: every article_id in orders should exist in articles

ri = {}
orders = dfs["orders"]; articles = dfs["articles"]; suppliers = dfs["suppliers"]; addresses = dfs["addresses"]

# checking if each "x_id" in df y exists in the df z
ri["orders->articles"]  = (~orders["article_id"].isin(articles["Article_ID"])).mean()
ri["orders->suppliers"] = (~orders["supplier_id"].isin(suppliers["supplier_id"])).mean()
ri["addresses->suppliers"] = (~addresses["supplier_id"].isin(suppliers["supplier_id"])).mean()

# fraction of missing references (0.0 = perfect). Multiply by 100 for %
{k: round(v*100,2) for k,v in ri.items()}


{'orders->articles': 100.0,
 'orders->suppliers': 100.0,
 'addresses->suppliers': 0.0}

#### Interpretation of referential integrity: 
- orders->articles: 100001 vs. "100001" don't align
- orders->suppliers: "1001" vs. 1001 don't align 

In [57]:
# 3) Missingness in key columns
key_cols = [
    ("orders", ["order_id","price_per_quantity", "quantity","order_value","supplier_id","article_id"]),
    ("articles", ["Article_ID","Article","Industry"]),
    ("addresses", ["supplier_id","country"]),
    ("suppliers", ["supplier_id","total_company_revenue","domain","status","certificates_valid"]),
    ("indices", ["country_id","human_rights_index","enivronmental_risk"]),
]
miss = []
for name, cols in key_cols:
    for c in cols:
        miss.append({"dataset": name, "column": c, "null_%": round(dfs[name][c].isna().mean()*100,1)})
pd.DataFrame(miss).sort_values(["null_%","dataset"], ascending=[False,True])

,dataset,column,null_%
13,suppliers,domain,100.0
17,indices,human_rights_index,43.0
18,indices,enivronmental_risk,11.6
9,addresses,supplier_id,0.0
10,addresses,country,0.0
6,articles,Article_ID,0.0
7,articles,Article,0.0
8,articles,Industry,0.0
16,indices,country_id,0.0
0,orders,order_id,0.0


In [58]:
# Check how missingness in indices columns is distributed

# Fix the column name typo first for consistency
dfs["indices"] = dfs["indices"].rename(columns={"enivronmental_risk": "environmental_risk"})

# Columns to check
cols_to_check = ["human_rights_index", "environmental_risk"]

missing_summary = {}

for col in cols_to_check:
    # Countries with missing values in this column
    missing_countries = dfs["indices"].loc[dfs["indices"][col].isna(), "country_id"]

    missing_summary[col] = {
        "missing_count": len(missing_countries),
        "unique_countries_missing": missing_countries.nunique(),
        "countries": sorted(missing_countries.unique().tolist())
    }

# Show as DataFrame for readability
pd.DataFrame(missing_summary).T


,missing_count,unique_countries_missing,countries
human_rights_index,107,107,"[ABW, AIA, ALA, AND, ASM, ATA, ATF, ATG, AUT, ..."
environmental_risk,29,29,"[ALA, ASM, ATA, ATF, BES, BLM, BVT, CUW, CXR, ..."


#### Interpretation of missingness of values: 
- suppliers.domain: can be ignored for this analysis
- indices.human_rights_index/.environmental_risk:
  --> seems to be very scattered across a large share of countries
  --> filling them with avg/median would introduce a lot of noise and bias
  --> approach: leave them as NaN in dataset and decide later in scoring whether to drop these suppliersor assign them as neutral

In [59]:
# 4) Value hygiene check (case, whitespace)
def levels(df, col): 
    return (df[col].value_counts(dropna=False).head(20))

print("articles.industry – top values:")
display(levels(dfs["articles"], "Industry"))

print("addresses.country – top values:")
display(levels(dfs["addresses"], "country"))

articles.industry – top values:


Industry
Plastics       3
Metals         2
Electronics    2
plastics       2
metals         1
electronics    1
Name: count, dtype: int64

addresses.country – top values:


country
China          41
Germany        26
Sweden         11
germany         8
switzerland     7
Switzerland     7
Israel          7
china           6
israel          4
Name: count, dtype: int64

#### Interpretation of value hygiene
-> .str.lower().str.strip() normalization is needed for industry and country

In [60]:
# 5) Numeric anomalies (negatives/zeros/outliers)

# a) orders df
o = dfs["orders"].copy()
anomalies = {
    "neg_qty": int((o["quantity"] < 0).sum()),
    "zero_qty": int((o["quantity"] == 0).sum()),
    "neg_price": int((o["price_per_quantity"] < 0).sum()),
    "neg_value": int((o["order_value"] < 0).sum()),
}
anomalies

{'neg_qty': 0, 'zero_qty': 0, 'neg_price': 0, 'neg_value': 0}

In [61]:
# b) suppliers df

s = dfs["suppliers"].copy()

# ensure numeric (in case it's parsed as string)
s["total_company_revenue_num"] = pd.to_numeric(s["total_company_revenue"], errors="coerce")

sup_anomalies = {
    "revenue_is_na": int(s["total_company_revenue_num"].isna().sum()),
    "revenue_negative": int((s["total_company_revenue_num"] < 0).sum()),
    "revenue_zero": int((s["total_company_revenue_num"] == 0).sum()),
}

sup_anomalies

{'revenue_is_na': 0, 'revenue_negative': 0, 'revenue_zero': 9}

#### Interpretation of numeric anomalies 
-> zero revenues: maybe inactive suppliers or bad data (decide later) 

## Step 3: Data Cleaning

In [63]:
dfs_clean = deepcopy(dfs)  # Work on a copy so we preserve raw data for reference

# 1) Fix column name typo in indices
if "enivronmental_risk" in dfs_clean["indices"].columns:
    dfs_clean["indices"] = dfs_clean["indices"].rename(columns={"enivronmental_risk": "environmental_risk"})

# 2) Lowercase column names for articles df to match other datasets
art = dfs_clean["articles"]                      # alias to avoid chained assignment warnings
art.columns = [c.strip().lower() for c in art.columns]

# 3) Standardize string categorical fields (lowercase → strip → title-case where relevant)
## Industry – lowercase, strip spaces for grouping
dfs_clean["articles"]["industry"] = dfs_clean["articles"]["industry"].str.strip().str.lower()

## Country – lowercase → strip → title-case for display
dfs_clean["addresses"]["country"] = (
    dfs_clean["addresses"]["country"]
    .str.strip()
    .str.lower()
    .str.title()
)

# Map to ISO3 codes for joining with indices
def country_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

dfs_clean["addresses"]["country_iso3"] = dfs_clean["addresses"]["country"].apply(country_to_iso3)

# 4) Deduplicate primary keys where possible
## suppliers – keep first occurrence
dfs_clean["suppliers"] = dfs_clean["suppliers"].drop_duplicates(subset=["supplier_id"], keep="first")


# # 5) Fix referential integrity issues (string ↔ int mismatches)
# ARTICLES: clean all object columns, then (optionally) coerce article_id to numeric
for col in art.select_dtypes(include="object").columns:
    art[col] = art[col].astype(str).str.strip().str.replace('"', '', regex=False)
art["article_id"] = pd.to_numeric(art["article_id"], errors="ignore")

# ORDERS: clean article_id and supplier_id so they can match other tables
ord_ = dfs_clean["orders"]
for col in ["article_id", "supplier_id"]:
    ord_[col] = ord_[col].astype(str).str.strip().str.replace('"', '', regex=False)
ord_[["article_id","supplier_id"]] = ord_[["article_id","supplier_id"]].apply(
    pd.to_numeric, errors="ignore"
)

# SUPPLIERS: normalize supplier_id
sup = dfs_clean["suppliers"]
sup["supplier_id"] = sup["supplier_id"].astype(str).str.strip().str.replace('"', '', regex=False)
sup["supplier_id"] = pd.to_numeric(sup["supplier_id"], errors="ignore")

# ADDRESSES: keep supplier_id consistent with suppliers/orders
addr = dfs_clean["addresses"]
addr["supplier_id"] = addr["supplier_id"].astype(str).str.strip().str.replace('"', '', regex=False)
addr["supplier_id"] = pd.to_numeric(addr["supplier_id"], errors="ignore")



# Preview the first 5 rows of each dataset and basic structure info
for name, df in dfs_clean.items():
    print(f"\n--- {name.upper()} ---")
    display(df.head(5))  
    print(df.info())


# --- Quick validation after cleaning ---
print("\n[Validation] Referential integrity after type fixes:")
ri_check = {
    "orders->articles": (~dfs_clean["orders"]["article_id"].isin(dfs_clean["articles"]["article_id"])).mean(),
    "orders->suppliers": (~dfs_clean["orders"]["supplier_id"].isin(dfs_clean["suppliers"]["supplier_id"])).mean(),
    "addresses->suppliers": (~dfs_clean["addresses"]["supplier_id"].isin(dfs_clean["suppliers"]["supplier_id"])).mean(),
}
{k: round(v*100, 2) for k, v in ri_check.items()}



--- ADDRESSES ---


,supplier_id,country,country_iso3
0,1003,Sweden,SWE
1,1084,Germany,DEU
2,1076,Germany,DEU
3,1093,Germany,DEU
4,1037,Germany,DEU


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   supplier_id   117 non-null    int64 
 1   country       117 non-null    object
 2   country_iso3  117 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.9+ KB
None

--- ARTICLES ---


,article_id,article,industry
0,1000001,Standardteile aus Aluminium,metals
1,1000002,Blechteile,metals
2,1000003,Rohmetalle,metals
3,1000004,Elektronische Standardteile,electronics
4,1000005,Technische Produkte,electronics


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article_id  11 non-null     int64 
 1   article     11 non-null     object
 2   industry    11 non-null     object
dtypes: int64(1), object(2)
memory usage: 392.0+ bytes
None

--- INDICES ---


...

,order_id,price_per_quantity,quantity,order_value,supplier_id,article_id
0,O00107,470.33,92,43270.36,1001,1000001
1,O00109,241.40,163,39348.20,1001,1000001
2,O00112,357.63,7,2503.41,1001,1000002
3,O00278,865.39,125,108173.75,1001,1000002
4,O00300,889.69,58,51602.02,1001,1000003


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            500 non-null    object 
 1   price_per_quantity  500 non-null    float64
 2   quantity            500 non-null    int64  
 3   order_value         500 non-null    float64
 4   supplier_id         500 non-null    int64  
 5   article_id          500 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 23.6+ KB
None

--- SUPPLIERS ---


,supplier_id,total_company_revenue,domain,status,certificates_valid
0,1003,1.377955e+09,NaN,inactive,no
1,1084,1.074170e+09,NaN,active,yes
2,1076,9.030552e+08,NaN,active,yes
3,1093,7.450531e+08,NaN,active,yes
4,1037,1.700478e+09,NaN,inactive,yes


<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 105
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   supplier_id            100 non-null    int64  
 1   total_company_revenue  100 non-null    float64
 2   domain                 0 non-null      float64
 3   status                 100 non-null    object 
 4   certificates_valid     100 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 4.7+ KB
None

[Validation] Referential integrity after type fixes:


{'orders->articles': 0.0,
 'orders->suppliers': 0.0,
 'addresses->suppliers': 0.0}

In [ ]:
# Next steps: 
# 3) Define Risk Taxonomy
# -> WICHTIG: HIERFÜR PPT ANSCHAUEN UND NOCHMAL AUFGABENSTELLUNG & TACTOS WEBSITE DURCHLESEN GENAU 
# 4) Feature engineering
# 5) Normalize each feature
# 6) Compose quantitative risk score
# 7) Build result table 
# 8) Insights & patterns 
# 9) Assumptions log
# 10) Next steps 
# 11) Slides preparation 

In [ ]:
# Cleaning: things from above + address column + environmental risk typo